##  Setting-up The Environment


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2 
from tqdm import tqdm 
import warnings 
warnings.filterwarnings("ignore")


In [ ]:
import os

train_path = "/kaggle/input/waste-classification-data/DATASET/TRAIN"
test_path = "/kaggle/input/waste-classification-data/DATASET/TEST"

# Check if the paths exist
train_exists = os.path.exists(train_path)
test_exists = os.path.exists(test_path)

print("Train path exists:", train_exists)
print("Test path exists:", test_exists)


## Preprocessing 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers.schedules as schedules
from tensorflow.keras.layers import Conv2D,MaxPooling2D , Activation , Dense , Flatten , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator , img_to_array , load_img
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.callbacks import EarlyStopping ,ModelCheckpoint
from glob import glob


## visualization 


In [ ]:
x_data = []
y_data = []
for category in glob(train_path+"/*"):
    for file in tqdm(glob(category+"/*")):
        img_array = cv2.imread(file)
        img_array = cv2.cvtColor(img_array,cv2.COLOR_BGR2RGB)
        x_data.append(img_array)
        y_data.append(category.split('/')[-1])

data = pd.DataFrame({'image': x_data , "label":y_data})

In [ ]:
data.shape

In [ ]:
print(data.columns)
print(type(data))
print(data.info())
print(data.isnull().sum())

In [ ]:
data.head(1)

In [ ]:
import matplotlib.pyplot as plt 

# Displaying 5 images to inspect dataset
plt.figure(figsize=(20,15))
for i in range(9):
    plt.subplot(4,3,(i%12)+1)
    index = np.random.randint(15000)
    plt.title("this is of {0}".format(data.label[index]))
    plt.imshow(data.image[index])
    

plt.tight_layout()

plt.show


In [ ]:
dimensions = data['image'].apply(lambda img: img.shape)
print(dimensions.value_counts())

In [ ]:
pixel_means = data['image'].apply(lambda img: img.mean())
pixel_stds = data['image'].apply(lambda img: img.std())

print("Mean pixel intensity:", pixel_means.mean())
print("Standard deviation of pixel intensity:", pixel_stds.mean())


In [ ]:
print(data['label'].value_counts())

#visulaizeing the class distribution 
data['label'].value_counts().plot(kind='bar',color='skyblue')
plt.title('class Distribution')
plt.xlabel("classes")
plt.ylabel("count")
plt.show()

In [ ]:
## trying to find any corrupt or null images in the dataset
missing_images = data['image'].isnull().sum()
print(f"number of missing images : {missing_images}")

corrupt_images = data['image'].apply(lambda img: img.size==0).sum()
print(f" number of corrupt files : {corrupt_images}")

## Model Preperation and Training


In [ ]:
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense , Conv2D, MaxPooling2D , Flatten , Dropout , Activation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator , img_to_array , load_img
from tensorflow.keras.utils import plot_model
from glob import glob


In [ ]:
# model archetecture code
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape=(224,224,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('sigmoid'))


optimizer= Adam(learning_rate=0.001)
model.compile(loss= "binary_crossentropy",
              optimizer = optimizer,
              metrics = ["accuracy"])



In [ ]:
model.summary()

In [ ]:

datagen = ImageDataGenerator(rescale = 1./255,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)


In [ ]:
## image generators definations 
## for image generations and rescaling b/w 0-255
train_generator = datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 256,
    color_mode = 'rgb',
    class_mode = 'categorical'
)

test_generator = datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 256,
    color_mode = 'rgb',
    class_mode = 'categorical'
)

In [ ]:
# model check points
checkpoint = ModelCheckpoint("best_model.keras", 
                             monitor="val_loss", 
                             save_best_only=True, 
                             mode="min", 
                             verbose=1)

In [ ]:

early_stopping = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
hist = model.fit(
    train_generator,
    epochs=20,
    validation_data = test_generator,
    callbacks=[early_stopping,checkpoint]
)